In [ ]:
import boto3
from boto3.session import Session
import asyncio
import boto3
import json
import sys

import os
from IPython.display import Markdown, display

In [ ]:

boto_session = Session()
region = boto_session.region_name

ssm_client = boto3.client('ssm', region_name=region)
agent_arn_response = ssm_client.get_parameter(Name='/agent_a3_wiki_agentcore/runtime/agent_arn')
agent_arn = agent_arn_response['Parameter']['Value']
print(f"Retrieved Agent ARN: {agent_arn}")

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)


In [ ]:


boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Me conte sobre os tipos de templates?"}),
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8", errors="replace")
            print(line)
            if line.startswith("data: "):
                line = line[6:]
                content.append(line)
    # join and display the streamed data lines as Markdown
    display(Markdown("\n".join(content)))
else:
    # Non-stream responses can be bytes, strings, or lists. Handle robustly.
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]

    # Take the first event for display; decode if bytes
    first = events[0] if events else ""
    if isinstance(first, bytes):
        text = first.decode("utf-8", errors="replace")
    else:
        # Convert other types to string safely
        try:
            text = str(first)
        except Exception as e:
            text = f"<unserializable event: {e}>"

    # Try to parse JSON; if parsing fails, show raw text inside a code block
    try:
        parsed = json.loads(text)
        pretty = json.dumps(parsed, ensure_ascii=False, indent=2)
        display(Markdown(f"```json\n{pretty}\n```"))
    except Exception:
        # Not valid JSON — show raw text for debugging
        display(Markdown("```text\n" + text + "\n```"))
    
